# 로지스틱 회귀분석 개요

## 로지스틱 회귀분석 특징
    이항 로지스틱 회귀 분석은 종속변수가 0과 1이며 베르누이 분포를 따를 경우 사용
    모델의 산출 값은 각 데이터가 1이 될 확률이며 이진 분류를 위해서 경계값(threshold)이 필요
    모델 평가를 위해 각종 분류 관련 지표 및 AUC 활용
    
## 승산비(OR, Odds Ratio)
    특정 독립변수를 제외한 나머지 값을 고정하고 해당 독립변수가 1 증가 시 변화하는 승산(odds)의 비
    
## statsmodels - Logit()
    로지스틱 회귀분석을 실시하는 statsmodels의 함수
    endog, exog 인자에 각각 종속변수와 독립변수를 할당
    산출 모델 객체의 params 어트리뷰트에 모델의 계수 저장
    산출 모델 객체의 predict() 메서드로 예측값을 생산하며 이는 종속변수가 1이 될 확률값

In [2]:
import pandas as pd
import numpy as np
from statsmodels.api import Logit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [4]:
df = pd.read_csv("iris.csv")
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa


In [5]:
df["Species"].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [6]:
df["is_setosa"] = (df["Species"] == "setosa") + 0
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,is_setosa
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1


In [7]:
model = Logit(endog = df["is_setosa"],
             exog=df.iloc[:,:2]).fit()
model

Optimization terminated successfully.
         Current function value: 0.036374
         Iterations 11


In [8]:
model.params

Sepal.Length    -7.529945
Sepal.Width     13.130734
dtype: float64

In [9]:
model.pvalues

Sepal.Length    0.000828
Sepal.Width     0.000989
dtype: float64

In [10]:
model.tvalues

Sepal.Length   -3.343109
Sepal.Width     3.293594
dtype: float64

In [11]:
pred = model.predict(df.iloc[:3,:2])
pred

0    0.999477
1    0.923824
2    0.998678
dtype: float64

In [12]:
(pred < 0.5) + 0

0    0
1    0
2    0
dtype: int32

## sklearn - LogisticRegression()
    로지스틱 회귀분석을 실시하는 sklearn의 함수
    fit_intercept, solver 인자로 절편 적합 여부 및 최적화 알고리즘 설정 가능
    random_state 인자에 자연수를 할당하여 결과 고정 가능
    fit() 메서드에 독립변수 및 종속변수 할당
    산출 모델 객체의 coef_ 어트리뷰트에 모델의 계수 저장
    산출 모델 객체의 predict_proba() 메서드로 예측값을 생산하며 두 번째 열이 종속변수가 1이 될 확률값

In [13]:
model = LogisticRegression(random_state=123)
model.fit(X=df.iloc[:,:2], y=df["is_setosa"])
model

LogisticRegression(random_state=123)

In [14]:
model.coef_

array([[-3.38829757,  3.1645277 ]])

In [16]:
model.intercept_  #절편

array([8.32330389])

In [17]:
pred = model.predict_proba(df.iloc[:3,:2])
pred

array([[0.10727976, 0.89272024],
       [0.22895365, 0.77104635],
       [0.07413821, 0.92586179]])

In [18]:
pred = pred[:,1]
(pred > 0.5) + 0

array([1, 1, 1])

## sklearn - roc_auc_score()
    AUC(Area Under Curve)를 산출하는 sklearn의 함수
    y_true, y_score 인자에 각각 종속변수와 예측 확률값 할당

In [19]:
pred = model.predict_proba(df.iloc[:,:2])
pred = pred[:,1]
pred[:10]

array([0.89272024, 0.77104635, 0.92586179, 0.92738323, 0.94126096,
       0.91436651, 0.97058885, 0.89484454, 0.93034007, 0.82210603])

In [20]:
from sklearn.metrics import roc_auc_score

In [21]:
roc_auc_score(y_true=df["is_setosa"], y_score=pred)

0.9999999999999999

## sklearn - accuracy_score()
    분류모델의 정확도를 산출하는 sklearn의 함수
    y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당
    
## sklearn - f1_score()
    분류모델의 f1 값을 산출하는 sklearn의 함수
    y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당
    
## sklearn - precision_score()
    분류델의 정밀도(precision)를 산출하는 sklearn의 함수
    y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당
    
 ## sklearn - recall_score()
     분류모델의 재현율(recall)를 산출하는 sklearn의 함수
     y_pred와 y_true에 각각 예측 분류 결과와 실제 값을 할당

In [23]:
accuracy_score(y_true=df["is_setosa"], y_pred=(pred>0.8)+0)

0.9466666666666667

문제 01. 독립변수를 혈압, 혈당, BMI, 인슐린으로 하고 종속변수를 당뇨 여부로 할 경우 분류 정확도는 얼마인가? statsmodels 함수 사용. 데이터는 학습:평가 = 8:2로 분리 후 계산

In [24]:
df = pd.read_csv("diabetes.csv")
df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [25]:
from sklearn.model_selection import train_test_split

In [26]:
df_train, df_test = train_test_split(df, train_size=0.8, random_state=123)
df_train.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
318,3,115,66,39,140,38.1,0.150,28,0
313,3,113,50,10,85,29.5,0.626,25,0


In [27]:
model = Logit(endog=df_train["Outcome"],
             exog=df_train.loc[:,["BloodPressure","Glucose","BMI","Insulin"]]).fit()

Optimization terminated successfully.
         Current function value: 0.626579
         Iterations 5


In [29]:
pred = model.predict(exog=df_test.loc[:,["BloodPressure","Glucose","BMI","Insulin"]])
pred[:4]

236    0.462956
395    0.507051
36     0.359735
210    0.314389
dtype: float64

In [30]:
pred_class = (pred>0.5)+0
pred_class[:4]

236    0
395    1
36     0
210    0
dtype: int32

In [32]:
accuracy_score(y_pred=pred_class, y_true=df_test["Outcome"])

0.7012987012987013

문제 02. 독립변수를 혈당, BMI, 나이로 하고 종속변수를 당뇨 여부로 할 경우 나이의 승산비는 얼마인가?

In [33]:
model = Logit(endog=df["Outcome"],
             exog=df.loc[:,["Glucose","BMI","Age"]]).fit()
model.params

Optimization terminated successfully.
         Current function value: 0.656276
         Iterations 4


Glucose    0.009368
BMI       -0.035639
Age       -0.012898
dtype: float64

In [34]:
np.exp(model.params)

Glucose    1.009412
BMI        0.964989
Age        0.987184
dtype: float64

문제 03. 독립변수를 혈당, BMI, 나이로 하고 종속변수를 당뇨 여부로 할 경우 모델의 AUC는 얼마인가?

In [36]:
model = Logit(endog=df["Outcome"],
             exog=df.loc[:,["Glucose","BMI","Age"]]).fit()

Optimization terminated successfully.
         Current function value: 0.656276
         Iterations 4


In [37]:
model.predict(df.loc[:,["Glucose","BMI","Age"]])[:5]

0    0.387961
1    0.365506
2    0.615678
3    0.392087
4    0.336654
dtype: float64

In [38]:
roc_auc_score(y_true=df["Outcome"],
             y_score=model.predict(df.loc[:, ["Glucose","BMI","Age"]]))

0.5414253731343283